High level idea: use satellite imagery to explore Georgia's rivers and
sink basins, and see how they have been affected over the years, particularly
when considering the build up of urban centers near these areas.

Some sources/links:
- https://www.americanrivers.org/conservation-resource/running-dry-flint/

In [22]:
import ee
import geemap
import pandas as pd
import geopandas as gpd

ee.Initialize()

def pd_shp_to_ee_poly(shp):
    """Converts Polygon from GeoPandas to a ee.Geometry.Polygon
    object suitable for use within Google Earth Engine."""
    xs, ys = shp.exterior.coords.xy
    shp_list = [[x, y] for x, y in zip(xs, ys)]
    roi = ee.Geometry.Polygon(shp_list, None, False)
    return roi


def get_county_roi(county_name):
    """Returns a ee.Geometry.Polygon object representing
    a particular county within Georgia, along with the
    centroid of that object."""
    ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
    county_shp = ga_counties[ga_counties["NAME10"] == county_name].geometry.values[0]
    xs, ys = county_shp.centroid.coords.xy
    county_roi = pd_shp_to_ee_poly(county_shp).simplify(maxError = 1)

    return county_roi, (xs[0], ys[0])


def union_polygons(poly_ls):
    basePoly = poly_ls[0]
    for poly in poly_ls[1:]:
        basePoly = basePoly.union(poly, maxError = 1)

    return basePoly


def landsat_roi(roi, bands, fname):
    collection = geemap.landsat_timeseries(
        roi = roi, 
        start_year = 1985, end_year = 2021, 
        start_date = '06-10', end_date = '09-20'
    )

    video_args = {
        'dimensions': 768,
        'region': roi,
        'framesPerSecond': 2,
        'bands': bands,
        'min': 0,
        'max': 4000,
        'gamma': [1, 1, 1]
    }

    geemap.download_ee_video(collection, video_args, fname)
    geemap.add_text_to_gif(
        fname, fname, xy=('92%', '3%'), 
        text_sequence = 1985, font_size = 20, 
        font_color = '#ffffff', 
        add_progress_bar = False
    )

county_list = [
    'Camden', 'Glynn', 'McIntosh', 'Liberty',
    'Bryan', 'Chatham'
]

ga_counties = gpd.read_file("ga-counties/Counties_Georgia.shp")
poly_list = []

for county in county_list:
    county_shp = ga_counties[ga_counties["NAME10"] == county].geometry.values[0]
    poly_list.append(pd_shp_to_ee_poly(county_shp).simplify(maxError = 1))

coast = union_polygons(poly_list).simplify(maxError = 1)

In [23]:
bands = ['Red', 'Green', 'Blue']
fname = 'vids/coast-plain.gif'
landsat_roi(coast, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/coast-plain.gif


![](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/coast-plain.gif)

In [24]:
bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/coast-geology.gif'
landsat_roi(coast, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/coast-geology.gif


![](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/coast-geology.gif)

In [25]:
# Lake Sidney Lanier
lake_sidney = ee.Geometry.Polygon(
    [[-84.1484070, 34.4408936],
    [-83.7007141, 34.4488212],
    [-83.6924744, 34.1072564],
    [-84.1525269, 34.1049822]], None, False
)

savannah_basin_1 = ee.Geometry.Polygon(
    [[-82.6158142, 33.8829573],
    [-82.0925903, 33.9194319],
    [-82.0582581, 33.5734381],
    [-82.5856018, 33.5242237]], None, False
)

savannah_basin_2 = ee.Geometry.Polygon(
    [[-83.1266785, 34.5993024],
    [-83.1307983, 34.2980684],
    [-82.6089478, 34.3264260],
    [-82.6295471, 34.6173871]], None, False
)

In [26]:
bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/lake-sidney-geology.gif'
landsat_roi(lake_sidney, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/lake-sidney-geology.gif


![](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/lake-sidney-geology.gif)

In [27]:
bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/savannah-basin-1-geology.gif'
landsat_roi(savannah_basin_1, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/savannah-basin-1-geology.gif


![sas](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/savannah-basin-1-geology.gif)

In [28]:
bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/savannah-basin-2-geology.gif'
landsat_roi(savannah_basin_2, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/savannah-basin-2-geology.gif


![](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/savannah-basin-2-geology.gif)

In [29]:
flint_river = ee.Geometry.Polygon(
    [[-84.9909210, 30.8933867],
    [-84.9943542, 30.6869354],
    [-84.5933533, 30.6774871],
    [-84.6063995, 30.9010464]], None, False
)

bands = ['SWIR2', 'SWIR1', 'Blue']
fname = 'vids/flint-geology.gif'
landsat_roi(savannah_basin_2, bands, fname)

Generating URL...
Please wait ...
The GIF image has been saved to: /mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/flint-geology.gif


![savannah_basin](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/flint-geology.gif)

In [33]:
collection = geemap.create_timeseries(
    collection = ee.ImageCollection('JRC/GSW1_3/MonthlyHistory'),
    start_date = "1984-03-16",
    end_date = "2021-01-01",
    region = flint_river,
    bands = "water",
    frequency = "month"
)

video_args = {
    'dimensions': 768,
    'region': flint_river,
    'framesPerSecond': 6,
    'bands': ['water'],
    'min': 0.0,
    'max': 2.0,
    'palette': ['ffffff', 'fffcb8', '0905ff'],
    'bestEffort': True
}

fname = "vids/flint-monthly-recurrence.gif"
geemap.download_ee_video(collection, video_args, fname)
# geemap.add_text_to_gif(
#     fname, fname, xy=('92%', '3%'), 
#     text_sequence = 1985, font_size = 20, 
#     font_color = '#ffffff', 
#     add_progress_bar = False
# )

Generating URL...
Total request size (145626624 pixels) must be less than or equal to 26214400 pixels.


![flint](/mnt/c/Users/adavi/Documents/comp-journal/ajc-satellite/vids/flint-monthly-recurrence.gif)